In [1]:
import os
import datetime
import polars as pl
import pandas as pd
import altair as alt
from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.zjisti_vazbu import zjisti_vazbu

pl.Config(tbl_rows=100)

In [2]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("rok") >= 1800)

df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["008","100_a","245_a","245_p"], keep="first")
print(len(df))

794838
705250


In [4]:
df = df.drop_nulls(subset=['700_a','700_7','700_4']).explode(['700_a','700_7','700_4'])

In [7]:
len(df)

412481

In [137]:
aut = pl.read_parquet(os.path.join("data","aut_vyber.parquet"))
cesi = aut.explode("370_c").filter(pl.col("370_c").str.contains("Česk")).explode("100_7").select(pl.col("100_7")).to_series().to_list()
len(cesi)

363993

## Budeme brát do úvahy pouze autorstvo vícenásobné!

In [9]:
vicenasobni = df.group_by('700_7').len().filter(pl.col('len') >= 3).select(pl.col('700_7')).to_series().to_list()
len(vicenasobni)

32395

In [11]:
df = df.filter(pl.col('700_7').is_in(vicenasobni))

In [13]:
len(df)

295355

## Kolik % lidí ten rok skončilo

In [89]:
df.sample(20)

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,655_ind2,655_a,655_7,655_2,655_ind1,655_x,655_z,655_y,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba,posledni_kniha
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str,i64
"""1""","""Mark, William,""","""jo20241218643""","[""aut""]","""1923-1996""",null,null,null,null,"""cpk19960126103""",""" nam a22 a 4500""","""961016s1996 xr e 0…","""(brož.) :""","[""Kč 17,90""]","[""80-237-3123-8""]",null,null,null,null,null,null,"""1""","""0""","""Marshale, pomozte!""",null,"""William Mark ; přeložil Tomáš …",null,null,null,null,null,"[""59 s. ;""]",null,"[""19 cm""]",null,null,null,"[""7""]","[""dobrodružné povídky""]","[""fd132059""]","[""czenas""]",null,null,null,null,"[""1""]","""Janeček, Tomáš""","""trl""",null,"""jx20050912021""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1996,59,"""brožovaná""",1996
"""1""","""Campbell, Alan,""","""kv2009540282""","[""aut""]","""1971-""",null,null,null,null,"""nkc20172877974""",""" cam a22 i 4500""","""161220s2016 xr g 0…","""(brožováno) :""","[""Kč 349,00""]","[""978-80-7193-408-0""]",null,null,null,null,null,null,"""1""","""0""","""Železný anděl""",null,"""Alan Campbell ; překlad: Roman…",null,null,null,null,null,"[""377 stran ;""]",null,"[""20 cm""]",null,null,null,"[""7"", ""7"", … ""9""]","[""skotské romány (anglicky)"", ""fantasy romány"", … ""fantasy novels""]","[""fd133439"", ""fd184199"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,"[""1""]","""Tilcer, Roman,""","""trl""","[""1987-""]","""xx0095591""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2016,377,"""brožovaná""",2025
"""1""","""Černý, Karel,""","""jk01021585""","[""aut""]","""1879-1955""",null,null,null,null,"""bk193500540""",""" nam a22 1 4500""","""990410s1935 xr …",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Injekční anaesthesie a narkosa…",null,"""napsali Karel Černý ... Fr. Ne…",null,null,null,null,null,"[""252 s. ;""]",null,"[""8°""]",null,null,null,null,null,null,null,null,null,null,null,"[""1""]","""Neuwirt, František,""","""aut""","[""1895-1957""]","""jk01090103""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1935,252,null,1971
"""1""","""Odincov, Viktor""",null,"[""aut""]",null,null,null,null,null,"""bk197701913""",""" nam a22 1 4500""","""970324s1976 xr e f |…",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Sociálně kulturní plánování a …",null,"""Viktor Odincov ; z rus. orig. …",null,null,null,null,null,"[""76 s. ;""]",null,"[""8°""]",null,null,null,"[""7""]","[""příručky""]","[""fd133209""]","[""czenas""]",null,null,null,null,"[""1""]","""Lindforsová, Milada""","""trl""",null,"""jk01072136""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1976,76,null,1989
"""1""","""Hanuš, Jaromír,""","""jx20051130021""","[""aut""]","""1954-""",null,null,null,null,"""cpk19960139023""",""" nam a22 a 4500""","""961206s1996 xr e l 0…","""(brož.) :""","[""Kč 160,00""]","[""80-85922-26-6""]",null,nul

In [16]:
df = df.join(df.group_by('700_7').agg(pl.col("rok").max().alias("posledni_kniha")), on="700_7", how="left")

In [18]:
df.filter(pl.col('rok') == pl.col('posledni_kniha'))

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,655_ind2,655_a,655_7,655_2,655_ind1,655_x,655_z,655_y,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba,posledni_kniha
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str,i64
"""1""","""Humphries, Sam,""","""xx0243080""","[""aut""]","""1977-""",null,null,null,null,"""nkc20223423198""",""" nam a22 i 4500""","""220629s2022 xr af g 6 0…","""(brožováno) :""","[""Kč 399,00""]","[""978-80-7595-577-7""]",null,null,null,null,null,null,"""1""","""0""","""Harley Quinn""",null,"""scénář Sam Humphries, Mark Rus…",null,"""Harley ničí vesmír""",null,null,null,"[""154 stran, 6 nečíslovaných stran obrazových příloh :""]","[""barevné ilustrace ;""]","[""26 cm""]",null,null,null,"[""7"", ""9""]","[""superhrdinské komiksy"", ""superhero comics""]","[""fd1057488"", null]","[""czenas"", ""eczenas""]",null,null,null,null,"[""1"", ""1"", … ""1""]","""Andolfo, Mirka,""","""ill""","[""1971-"", ""1979-"", … ""1987-""]","""xx0237257""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022,154,"""brožovaná""",2022
"""1""","""Humphries, Sam,""","""xx0243080""","[""aut""]","""1977-""",null,null,null,null,"""nkc20223423198""",""" nam a22 i 4500""","""220629s2022 xr af g 6 0…","""(brožováno) :""","[""Kč 399,00""]","[""978-80-7595-577-7""]",null,null,null,null,null,null,"""1""","""0""","""Harley Quinn""",null,"""scénář Sam Humphries, Mark Rus…",null,"""Harley ničí vesmír""",null,null,null,"[""154 stran, 6 nečíslovaných stran obrazových příloh :""]","[""barevné ilustrace ;""]","[""26 cm""]",null,null,null,"[""7"", ""9""]","[""superhrdinské komiksy"", ""superhero comics""]","[""fd1057488"", null]","[""czenas"", ""eczenas""]",null,null,null,null,"[""1"", ""1"", … ""1""]","""Timms, John""","""ill""","[""1971-"", ""1979-"", … ""1987-""]","""kv2016935319""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022,154,"""brožovaná""",2022
"""1""","""Humphries, Sam,""","""xx0243080""","[""aut""]","""1977-""",null,null,null,null,"""nkc20223423198""",""" nam a22 i 4500""","""220629s2022 xr af g 6 0…","""(brožováno) :""","[""Kč 399,00""]","[""978-80-7595-577-7""]",null,null,null,null,null,null,"""1""","""0""","""Harley Quinn""",null,"""scénář Sam Humphries, Mark Rus…",null,"""Harley ničí vesmír""",null,null,null,"[""154 stran, 6 nečíslovaných stran obrazových příloh :""]","[""barevné ilustrace ;""]","[""26 cm""]",null,null,null,"[""7"", ""9""]","[""superhrdinské komiksy"", ""superhero comics""]","[""fd1057488"", null]","[""czenas"", ""eczenas""]",null,null,null,null,"[""1"", ""1"", … ""1""]","""McCrea, John,""","""ill""","[""1971-"", ""1979-"", … ""1987-""]","""xx0025725""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022,154,"""brožovaná""",2022
"""1""","""Humphries, Sam,""","""xx0243080""","[""aut""]","""1977-""",null,null,null,null,"""nkc20223423198""",""" nam a22 i 4500""","""220629s2022 xr af g 6 0…","""(brožováno) :""","[""Kč 399,00""]","[""978-80-7595-577-7""]",null,null,null,null,null,null,"""1""","""0""","""Harley Quinn""",null,"""scénář Sam Humphries, Mar

In [139]:
podil_koncicich = df.filter(pl.col('700_7').is_in(cesi)).filter(pl.col('rok') == pl.col('posledni_kniha')).group_by('rok').agg(pl.col('700_7').n_unique()).rename({'700_7':'skoncilo'}).join(df.filter(pl.col('700_7').is_in(cesi)).group_by('rok').agg(pl.col('700_7').n_unique()).rename({'700_7':'celkem'}), on='rok', how='left').with_columns((pl.col('skoncilo') / pl.col('celkem')).alias('podil')).sort(by='rok')
podil_koncicich.tail(60)

rok,skoncilo,celkem,podil
i64,u32,u32,f64
1966,83,1270,0.065354
1967,81,1228,0.065961
1968,93,1171,0.079419
1969,101,1173,0.086104
1970,122,1191,0.102435
1971,114,1192,0.095638
1972,84,1262,0.066561
1973,90,1224,0.073529
1974,92,1331,0.069121


In [141]:
podil_koncicich_aut = df.filter(pl.col("700_4") == "aut").filter(pl.col('700_7').is_in(cesi)).filter(pl.col('rok') == pl.col('posledni_kniha')).group_by('rok').agg(pl.col('700_7').n_unique()).rename({'700_7':'skoncilo_aut'}).join(df.filter(pl.col('700_7').is_in(cesi)).filter(pl.col("700_4") == "aut").group_by('rok').agg(pl.col('700_7').n_unique()).rename({'700_7':'celkem_aut'}), on='rok', how='left').with_columns((pl.col('skoncilo_aut') / pl.col('celkem_aut')).alias('podil_aut')).sort(by='rok')
podil_koncicich_aut.tail(60)

rok,skoncilo_aut,celkem_aut,podil_aut
i64,u32,u32,f64
1966,44,540,0.081481
1967,42,547,0.076782
1968,46,485,0.094845
1969,63,533,0.118199
1970,77,520,0.148077
1971,70,536,0.130597
1972,59,552,0.106884
1973,48,522,0.091954
1974,42,573,0.073298


In [143]:
podil_koncicich_trl = df.filter(pl.col('700_7').is_in(cesi)).filter(pl.col("700_4") == "trl").filter(pl.col('rok') == pl.col('posledni_kniha')).group_by('rok').agg(pl.col('700_7').n_unique()).rename({'700_7':'skoncilo_trl'}).join(df.filter(pl.col('700_7').is_in(cesi)).filter(pl.col("700_4") == "trl").group_by('rok').agg(pl.col('700_7').n_unique()).rename({'700_7':'celkem_trl'}), on='rok', how='left').with_columns((pl.col('skoncilo_trl') / pl.col('celkem_trl')).alias('podil_trl')).sort(by='rok')
podil_koncicich_trl.tail(60)

rok,skoncilo_trl,celkem_trl,podil_trl
i64,u32,u32,f64
1966,12,275,0.043636
1967,11,251,0.043825
1968,14,250,0.056
1969,9,231,0.038961
1970,12,243,0.049383
1971,17,257,0.066148
1972,3,237,0.012658
1973,13,254,0.051181
1974,15,279,0.053763


In [145]:
podil_koncicich_ill = df.filter(pl.col('700_7').is_in(cesi)).filter(pl.col("700_4") == "ill").filter(pl.col('rok') == pl.col('posledni_kniha')).group_by('rok').agg(pl.col('700_7').n_unique()).rename({'700_7':'skoncilo_ill'}).join(df.filter(pl.col('700_7').is_in(cesi)).filter(pl.col("700_4") == "ill").group_by('rok').agg(pl.col('700_7').n_unique()).rename({'700_7':'celkem_ill'}), on='rok', how='left').with_columns((pl.col('skoncilo_ill') / pl.col('celkem_ill')).alias('podil_ill')).sort(by='rok')
podil_koncicich_ill.tail(60)

rok,skoncilo_ill,celkem_ill,podil_ill
i64,u32,u32,f64
1966,3,149,0.020134
1967,1,130,0.007692
1968,6,163,0.03681
1969,4,156,0.025641
1970,8,155,0.051613
1971,3,150,0.02
1972,4,176,0.022727
1973,3,160,0.01875
1974,5,155,0.032258


In [147]:
podil_koncicich = podil_koncicich.join(podil_koncicich_ill.select(pl.col(["rok","podil_ill"])),on="rok").join(podil_koncicich_trl.select(pl.col(["rok","podil_trl"])),on="rok").join(podil_koncicich_aut.select(pl.col(["rok","podil_aut"])),on="rok")

In [133]:
podil_koncicich.filter(pl.col('rok').is_between(1900,2000)).sort(by='podil',descending=True)

rok,skoncilo,celkem,podil,podil_ill,podil_trl,podil_aut
i64,u32,u32,f64,f64,f64,f64
1990,400,1949,0.205233,0.115299,0.146179,0.325347
1989,393,1968,0.199695,0.095455,0.15035,0.289855
1948,92,522,0.176245,0.093333,0.128205,0.248555
1944,40,227,0.176211,0.038462,0.192308,0.227848
1987,312,1920,0.1625,0.051095,0.089783,0.245238
1991,319,1970,0.161929,0.114537,0.111437,0.226391
1988,307,1930,0.159067,0.068807,0.121037,0.228113
1939,54,346,0.156069,0.078125,0.113924,0.166667
1938,57,378,0.150794,0.055556,0.19,0.157025


In [149]:
podil_koncicich.write_parquet(os.path.join('data','podily_koncicich_lidi_cs.parquet'))

## Nejproduktivnější autorstvo nevydávané po historických zlomech

In [34]:
def nevysli_po(rok, obdobi):
    vysli_pred = df.filter(pl.col("rok").is_between(rok-obdobi,rok)).select(pl.col("700_7")).to_series().to_list()
    vysli_po = set(df.filter(pl.col("rok") > rok).select(pl.col("700_7")).to_series().to_list())
    ukonceni = [v for v in vysli_pred if v not in vysli_po]
    print(f"Filtru vyhovuje {len(ukonceni)} lidí.")
    return df.filter(pl.col("700_7").is_in(ukonceni)).group_by(["700_a","700_7","700_4"]).len().sort(by="len",descending=True).head(100)

In [36]:
nevysli_po(1918, 18)

Filtru vyhovuje 579 lidí.


700_a,700_7,700_4,len
str,str,str,u32
"""Šulc, Pavel J.,""","""jk01131682""","""trl""",36
"""Sokol, Josef,""","""jk01120342""","""aut""",25
"""Dobeš, Jan,""","""jk01022405""","""ill""",24
"""Lepař, Jan,""","""jk01071891""","""aut""",23
"""Kádner, Karel,""","""jk01052486""","""trl""",19
"""Petrů, Václav,""","""jk01092851""","""trl""",18
"""Nečas, Jan Evangelista,""","""jk01082827""","""trl""",18
"""Hofírek, Ignác,""","""jk01041537""","""trl""",14
"""Blažek, František Sal.,""","""jk01012295""","""trl""",11


In [38]:
nevysli_po(1945, 5)

Filtru vyhovuje 442 lidí.


700_a,700_7,700_4,len
str,str,str,u32
"""Starý, Emilian,""","""ola200207398""","""cmp""",50
"""Kočí, Josef,""","""jk01061011""","""ill""",49
"""Adámek, Rudolf,""","""jk01010090""","""ill""",48
"""Řehořovský, Václav Karel,""","""jk01110057""","""aut""",25
"""Stříž, Antonín Ludvík,""","""jk01121563""","""trl""",23
"""Vilikovský, Václav,""","""jk01142373""","""aut""",23
"""Horčička, Josef,""","""jk01042166""","""aut""",22
"""Štvánová, Božena""","""jx20040908013""","""aut""",20
"""Slabý, František,""","""jk01111699""","""ill""",18


In [40]:
nevysli_po(1989, 25)

Filtru vyhovuje 18302 lidí.


700_a,700_7,700_4,len
str,str,str,u32
"""Kratochvíl, Zdeněk,""","""jk01063016""","""ill""",76
"""Ptáčková-Macháčková, Olga,""","""jk01101650""","""trl""",54
"""Procházka, František,""","""jk01101125""","""edt""",54
"""Ušák, Otto,""","""jk01140916""","""ill""",50
"""Müller, Karel,""","""jk01082438""","""ill""",45
"""Hájek, Bohumil,""","""jk01033032""","""aut""",44
"""Petrůj, Svatopluk,""","""jk01092852""","""aut""",41
"""Taufer, Jiří,""","""jk01132099""","""trl""",41
"""Janská, Anna""","""jk01051138""","""aut""",40


In [42]:
nevysli_po(2000, 9)

Filtru vyhovuje 12157 lidí.


700_a,700_7,700_4,len
str,str,str,u32
"""Thuma, Karel Ladislav,""","""jk01132366""","""ill""",169
"""Kárász, Karel,""","""jx20040906007""","""art""",136
"""Sychra, Pavel""","""jx20031204025""","""trl""",122
"""Vorel, Josef,""","""jk01150688""","""trl""",98
"""Schuster, R. K.""","""jx20040604008""","""trl""",95
"""Pošmurný, Oldřich,""","""xx0008143""","""art""",83
"""Novotný, Václav Juda,""","""jk01090778""","""trl""",80
"""Podolská, Zora""","""jx20040216029""","""trl""",67
"""Zábranský, Adolf,""","""jk01151856""","""ill""",60


## Normalizace

363993

Nejdříve porovnáme všechna x-letá období ve 20. století.

In [54]:
delky_pauzy = [5,10,15,20]
profese = ['ill','aut','trl']
odmlceni = []
for r in range(1820,2021):
    rocni = {'rok': r}
    for delka_pauzy in delky_pauzy:
        print(f"{r} + {delka_pauzy} let")
        vychazeli_pred = set(df.filter(pl.col("rok") < r).select(pl.col("700_7")).to_series().to_list())
        vychazeli_po = set(df.filter(pl.col("rok") > (r + delka_pauzy)).select(pl.col("700_7")).to_series().to_list())
        vychazeli_mezi = set(df.filter(pl.col("rok").is_between(r, r+delka_pauzy)).select(pl.col("700_7")).to_series().to_list())
        cancellovani = vychazeli_pred.intersection(vychazeli_po).difference(vychazeli_mezi)
        rocni[f'podil_vsech_{delka_pauzy}'] =  len(cancellovani) / len(vychazeli_pred)
        vychazeli_pred = set(df.filter(pl.col("rok") < r).filter(pl.col('700_7').is_in(cesi)).select(pl.col("700_7")).to_series().to_list())
        vychazeli_po = set(df.filter(pl.col("rok") > (r + delka_pauzy)).filter(pl.col('700_7').is_in(cesi)).select(pl.col("700_7")).to_series().to_list())
        vychazeli_mezi = set(df.filter(pl.col("rok").is_between(r, r+delka_pauzy)).filter(pl.col('700_7').is_in(cesi)).select(pl.col("700_7")).to_series().to_list())
        cancellovani = vychazeli_pred.intersection(vychazeli_po).difference(vychazeli_mezi)
        rocni[f'podil_cechu_{delka_pauzy}'] =  len(cancellovani) / len(vychazeli_pred)
        for prof in profese:
            try:
                print(prof)
                vychazeli_pred = set(df.filter(pl.col("rok") < r).filter(pl.col('700_7').is_in(cesi)).filter(pl.col('700_4') == prof).select(pl.col("700_7")).to_series().to_list())
                vychazeli_po = set(df.filter(pl.col("rok") > (r + delka_pauzy)).filter(pl.col('700_7').is_in(cesi)).filter(pl.col('700_4')  == prof).select(pl.col("700_7")).to_series().to_list())
                vychazeli_mezi = set(df.filter(pl.col("rok").is_between(r, r+delka_pauzy)).filter(pl.col('700_7').is_in(cesi)).filter(pl.col('700_4')  == prof).select(pl.col("700_7")).to_series().to_list())
                cancellovani = vychazeli_pred.intersection(vychazeli_po).difference(vychazeli_mezi)
                rocni[f'podil_cechu_{prof}_{delka_pauzy}'] =  len(cancellovani) / len(vychazeli_pred)
            except:
                pass
    odmlceni.append(rocni)

1820 + 5 let
ill
aut
trl
1820 + 10 let
ill
aut
trl
1820 + 15 let
ill
aut
trl
1820 + 20 let
ill
aut
trl
1821 + 5 let
ill
aut
trl
1821 + 10 let
ill
aut
trl
1821 + 15 let
ill
aut
trl
1821 + 20 let
ill
aut
trl
1822 + 5 let
ill
aut
trl
1822 + 10 let
ill
aut
trl
1822 + 15 let
ill
aut
trl
1822 + 20 let
ill
aut
trl
1823 + 5 let
ill
aut
trl
1823 + 10 let
ill
aut
trl
1823 + 15 let
ill
aut
trl
1823 + 20 let
ill
aut
trl
1824 + 5 let
ill
aut
trl
1824 + 10 let
ill
aut
trl
1824 + 15 let
ill
aut
trl
1824 + 20 let
ill
aut
trl
1825 + 5 let
ill
aut
trl
1825 + 10 let
ill
aut
trl
1825 + 15 let
ill
aut
trl
1825 + 20 let
ill
aut
trl
1826 + 5 let
ill
aut
trl
1826 + 10 let
ill
aut
trl
1826 + 15 let
ill
aut
trl
1826 + 20 let
ill
aut
trl
1827 + 5 let
ill
aut
trl
1827 + 10 let
ill
aut
trl
1827 + 15 let
ill
aut
trl
1827 + 20 let
ill
aut
trl
1828 + 5 let
ill
aut
trl
1828 + 10 let
ill
aut
trl
1828 + 15 let
ill
aut
trl
1828 + 20 let
ill
aut
trl
1829 + 5 let
ill
aut
trl
1829 + 10 let
ill
aut
trl
1829 + 15 let
ill
aut


In [63]:
odmlceni

rok,podil_vsech_5,podil_cechu_5,podil_cechu_aut_5,podil_cechu_trl_5,podil_vsech_10,podil_cechu_10,podil_cechu_aut_10,podil_cechu_trl_10,podil_vsech_15,podil_cechu_15,podil_cechu_aut_15,podil_cechu_trl_15,podil_vsech_20,podil_cechu_20,podil_cechu_aut_20,podil_cechu_trl_20,podil_cechu_ill_5,podil_cechu_ill_10,podil_cechu_ill_15,podil_cechu_ill_20
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1820-01-01 00:00:00,0.4,0.4,0.0,0.318182,0.28,0.28,0.0,0.272727,0.28,0.28,0.0,0.272727,0.12,0.12,0.0,0.136364,null,null,null,null
1821-01-01 00:00:00,0.357143,0.357143,0.0,0.333333,0.285714,0.285714,0.0,0.291667,0.107143,0.107143,0.0,0.166667,0.071429,0.071429,0.0,0.125,null,null,null,null
1822-01-01 00:00:00,0.344828,0.344828,0.0,0.346154,0.275862,0.275862,0.0,0.307692,0.103448,0.103448,0.0,0.153846,0.068966,0.068966,0.0,0.115385,null,null,null,null
1823-01-01 00:00:00,0.354839,0.354839,0.0,0.333333,0.290323,0.290323,0.0,0.296296,0.129032,0.129032,0.0,0.148148,0.096774,0.096774,0.0,0.111111,null,null,null,null
1824-01-01 00:00:00,0.363636,0.363636,0.5,0.310345,0.272727,0.272727,0.5,0.241379,0.121212,0.121212,0.5,0.137931,0.060606,0.060606,0.5,0.103448,null,null,null,null
1825-01-01 00:00:00,0.368421,0.342857,0.333333,0.266667,0.342105,0.314286,0.333333,0.233333,0.210526,0.171429,0.333333,0.133333,0.131579,0.085714,0.333333,0.1,null,null,null,null
1826-01-01 00:00:00,0.409091,0.410256,0.333333,0.323529,0.272727,0.25641,0.333333,0.205882,0.181818,0.153846,0.333333,0.147059,0.113636,0.076923,0.333333,0.117647,null,null,null,null
1827-01-01 00:00:00,0.466667,0.45,0.333333,0.323529,0.333333,0.3,0.333333,0.205882,0.222222,0.175,0.333333,0.117647,0.177778,0.125,0.333333,0.117647,null,null,null,null
1828-01-01 00:00:00,0.58,0.534884,0.333333,0.351351,0.4,0.325581,0.333333,0.189189,0.32,0.232558,0.333333,0.108108,0.26,0.162791,0.333333,0.108108,null,null,null,null


In [315]:
pl.DataFrame(odmlceni)

rok,podil_vsech_5,podil_cechu_5,podil_vsech_10,podil_cechu_10,podil_vsech_15,podil_cechu_15,podil_vsech_20,podil_cechu_20
i64,f64,f64,f64,f64,f64,f64,f64,f64
1820,0.477778,0.363636,0.422222,0.327273,0.377778,0.290909,0.288889,0.2
1821,0.494845,0.409836,0.43299,0.344262,0.350515,0.262295,0.28866,0.213115
1822,0.523364,0.457143,0.420561,0.357143,0.327103,0.257143,0.271028,0.214286
1823,0.5,0.425,0.398305,0.325,0.305085,0.2375,0.20339,0.15
1824,0.473684,0.402174,0.368421,0.293478,0.308271,0.23913,0.210526,0.163043
1825,0.492857,0.419355,0.407143,0.322581,0.314286,0.236559,0.214286,0.16129
1826,0.543624,0.459184,0.436242,0.336735,0.342282,0.265306,0.261745,0.193878
1827,0.529412,0.435644,0.411765,0.306931,0.326797,0.247525,0.267974,0.188119
1828,0.56875,0.466667,0.45,0.352381,0.2875,0.2,0.275,0.180952


In [55]:
pl.DataFrame(odmlceni).with_columns(
                   pl.col("rok").map_elements(
                       lambda x: datetime.date(year=int(x), month=1, day=1), 
                       return_dtype=pl.Date
                   ).cast(pl.Datetime)).write_parquet(os.path.join('data',f'podil_lidi_s_pauzou.parquet'))

In [56]:
odmlceni = pl.DataFrame(odmlceni).with_columns(
                   pl.col("rok").map_elements(
                       lambda x: datetime.date(year=int(x), month=1, day=1), 
                       return_dtype=pl.Date
                   ).cast(pl.Datetime))

## Znormalizovaní

In [65]:
pred_normalizaci = set(df.filter(pl.col("rok") <= 1972).select(pl.col("700_7")).to_series().to_list())
po_normalizaci = set(df.filter(pl.col("rok") >= 1988).select(pl.col("700_7")).to_series().to_list())
behem_normalizace = set(df.filter(pl.col("rok").is_between(1973,1987)).select(pl.col("700_7")).to_series().to_list())

In [67]:
znormalizovani = pred_normalizaci.intersection(po_normalizaci).difference(behem_normalizace)

In [71]:
df.filter(pl.col("700_7").is_in(znormalizovani) & pl.col("700_7").is_in(cesi)).group_by(["700_a",'700_7','700_4']).len().sort(by='len',descending=True)

700_a,700_7,700_4,len
str,str,str,u32
"""Thuma, Karel Ladislav,""","""jk01132366""","""ill""",169
"""Wowk, Jiří,""","""jk01151805""","""ill""",138
"""Heller, František""","""jk01040663""","""trl""",135
"""Weinfurter, Karel,""","""jk01151502""","""trl""",135
"""Hudeček, Stanislav,""","""jk01043215""","""ill""",104
"""Dvorský, František,""","""jk01030012""","""edt""",101
"""Krausová-Lesná, Milada,""","""jk01071925""","""trl""",98
"""Vorel, Josef,""","""jk01150688""","""trl""",98
"""Unzeitig, Vítězslav,""","""jk01140768""","""trl""",94


In [73]:
df.filter(pl.col("700_7").is_in(znormalizovani)).group_by(["700_a",'700_7','700_4']).len().sort(by='len',descending=True)

700_a,700_7,700_4,len
str,str,str,u32
"""Thuma, Karel Ladislav,""","""jk01132366""","""ill""",169
"""Wowk, Jiří,""","""jk01151805""","""ill""",138
"""Weinfurter, Karel,""","""jk01151502""","""trl""",135
"""Heller, František""","""jk01040663""","""trl""",135
"""Hudeček, Stanislav,""","""jk01043215""","""ill""",104
"""Dvorský, František,""","""jk01030012""","""edt""",101
"""Krausová-Lesná, Milada,""","""jk01071925""","""trl""",98
"""Vorel, Josef,""","""jk01150688""","""trl""",98
"""Unzeitig, Vítězslav,""","""jk01140768""","""trl""",94


## Zúnorovaní

In [75]:
pred_unorem = set(df.filter(pl.col("rok") <= 1948).select(pl.col("700_7")).to_series().to_list())
po_revoluci = set(df.filter(pl.col("rok") >= 1988).select(pl.col("700_7")).to_series().to_list())
v_sedesatkach = set(df.filter(pl.col("rok").is_between(1960,1969)).select(pl.col("700_7")).to_series().to_list())
behem_socialismu = set(df.filter(pl.col("rok").is_between(1949,1987)).select(pl.col("700_7")).to_series().to_list())
behem_padesatek = set(df.filter(pl.col("rok").is_between(1949,1959)).select(pl.col("700_7")).to_series().to_list())

In [76]:
zunorovani = pred_unorem.intersection(po_revoluci).difference(behem_socialismu)
df.filter(pl.col("700_7").is_in(zunorovani)).group_by(["700_a",'700_7']).len().sort(by='len',descending=True)

700_a,700_7,len
str,str,u32
"""Thuma, Karel Ladislav,""","""jk01132366""",169
"""Weinfurter, Karel,""","""jk01151502""",146
"""Wowk, Jiří,""","""jk01151805""",139
"""Dvorský, František,""","""jk01030012""",116
"""Hudeček, Stanislav,""","""jk01043215""",104
"""Vorel, Josef,""","""jk01150688""",99
"""Cihelka, Oldřich,""","""jk01020681""",88
"""Čutta, Václav,""","""jk01021972""",86
"""Wenig, Josef,""","""jk01151583""",84


In [79]:
df.filter(pl.col("700_7").is_in(zunorovani) & pl.col('700_7').is_in(cesi)).group_by(["700_a",'700_7']).len().sort(by='len',descending=True)

700_a,700_7,len
str,str,u32
"""Thuma, Karel Ladislav,""","""jk01132366""",169
"""Weinfurter, Karel,""","""jk01151502""",146
"""Wowk, Jiří,""","""jk01151805""",139
"""Dvorský, František,""","""jk01030012""",116
"""Hudeček, Stanislav,""","""jk01043215""",104
"""Vorel, Josef,""","""jk01150688""",99
"""Cihelka, Oldřich,""","""jk01020681""",88
"""Čutta, Václav,""","""jk01021972""",86
"""Wenig, Josef,""","""jk01151583""",84


In [83]:
zunorovani2 = pred_unorem.intersection(v_sedesatkach).difference(behem_padesatek)
df.filter(pl.col("700_7").is_in(zunorovani2)).group_by(["700_a",'700_7','700_4']).len().sort(by='len',descending=True)

700_a,700_7,700_4,len
str,str,str,u32
"""Heller, František""","""jk01040663""","""trl""",135
"""Nekovařík, Bohumil Zdeněk,""","""jk01082997""","""trl""",94
"""Shakespeare, William,""","""jn19981002129""","""ant""",61
"""Novák, Miloš,""","""jk01090489""","""ill""",44
"""Černý, Václav A.,""","""jk01021640""","""trl""",40
"""Heyduk, Josef,""","""jk01041005""","""trl""",39
"""Sodoma, Viktor,""","""ola2004219882""","""ill""",37
"""Renč, Václav,""","""jk01102340""","""trl""",35
"""Fischer, Jan,""","""jn19981228074""","""ill""",32
